In [96]:
import pandas as pd
import numpy as np
import os 


In [97]:
cwd = os.getcwd()

In [100]:
static = pd.read_pickle(cwd + '/raw_data/static_variable_state.pkl')
pop = static[['fips', 'Population']]

In [101]:
hosp = pd.read_pickle(cwd + '/processed_data/hospitalization_daily_state.pkl')
hosp['Week'] = hosp['Date'].dt.to_period('W')
hosp_week = hosp.groupby(['state_name', 'Variable', 'fips', 'Week', 'Description']).sum().reset_index()
hosp_week = hosp_week.merge(pop)
hosp_week['hospitalization_per_100k'] = (hosp_week['value']/hosp_week['Population'])*100000
hosp_week = hosp_week.drop(columns = {'Variable', 'Description'})
hosp_week['Week_start'] = hosp_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week['Week_end'] = hosp_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week = hosp_week.rename(columns = {'value' : 'hospitalization'})

/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_29365/3509825895.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  hosp_week = hosp.groupby(['state_name', 'Variable', 'fips', 'Week', 'Description']).sum().reset_index()


### Week1

In [102]:
### Smooth hosp
for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'hospitalization_per_100k_sm'] = \
    hosp_week[hosp_week['fips'] == fips]['hospitalization_per_100k'].rolling(window = 3).mean()

In [103]:
for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'hospitalization_per_100k_sm_lag1'] = \
    hosp_week[hosp_week['fips'] == fips]['hospitalization_per_100k_sm'].shift(1)

In [104]:
hosp_week = hosp_week.dropna().reset_index().drop(columns = ['index'])

In [105]:
hosp_week['Abs_Change_w1'] = hosp_week['hospitalization_per_100k'] - hosp_week['hospitalization_per_100k_sm_lag1']

In [106]:
hosp_week['Abs_Change_w1'] = round(hosp_week['Abs_Change_w1'], 2)

In [107]:
'''
Define threshold
'''
# hosp_week['Change'] = (hosp_week['hospitalization_per_100k'] - hosp_week['hospitalization_per_100k_lag1'])\
# /hosp_week['hospitalization_per_100k_lag1']


index_stable = hosp_week[(hosp_week['Abs_Change_w1'] < 1)&(hosp_week['Abs_Change_w1'] >= -1)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Stable'
index_stable = hosp_week[(hosp_week['Abs_Change_w1'] >= 1)&(hosp_week['Abs_Change_w1'] < 3)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Moderate Increasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w1'] >= 3)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Substantial Increasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w1'] > -3)&(hosp_week['Abs_Change_w1'] <= -1)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Moderate Decreasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w1'] <= -3)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Substantial Decreasing'


In [108]:
for index, row in hosp_week[hosp_week['Week_end'] >= '2021-01-01'].iterrows():
    prompt = "The most recent hospitalization trends are: " + hosp_week.loc[index-4, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-3, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-2, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-1, 'Abs_Trend'].upper() + ', and ' + hosp_week.loc[index, 'Abs_Trend'].upper()
    hosp_week.loc[index, 'Trend_prompt'] = prompt

In [109]:
for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'Abs_Trend_1w'] = hosp_week.loc[index, 'Abs_Trend'].shift(-1)


In [110]:
hosp_week = hosp_week.rename(columns = {'Abs_Change_w1' : 'Abs_Change'})
hosp_week = hosp_week.rename(columns = {'Abs_Trend_1w' : 'Abs_Trend_target_1w'})

In [111]:
hosp_week.columns

Index(['state_name', 'fips', 'Week', 'hospitalization', 'Population',
       'hospitalization_per_100k', 'Week_start', 'Week_end',
       'hospitalization_per_100k_sm', 'hospitalization_per_100k_sm_lag1',
       'Abs_Change', 'Abs_Trend', 'Trend_prompt', 'Abs_Trend_target_1w'],
      dtype='object')

In [112]:
ref.columns

Index(['state_name', 'fips', 'Week', 'Week_start', 'Week_end',
       'hospitalization_per_100k', 'hospitalization_per_100k_sm',
       'reported_cases_per_100k', 'Dose1_Pop_Pct', 'Series_Complete_Pop_Pct',
       'Additional_Doses_Vax_Pct', 'Abs_Change', 'Abs_Trend',
       'reported_cases_per_100k_gpt_trend',
       'hospitalization_per_100k_gpt_trend', 'Trend_prompt', 'dynamic_prompt',
       'dynamic_prompt_v2', 'dynamic_prompt_v3', 'week_id', 'uid',
       'Population', 'under_20', 'over_65', 'White', 'Black', 'Multiple_race',
       'Not_Hispanic', 'Hispanic', 'medicaid_coverage', 'medicare_coverage',
       'uninsured_percent', 'medicaid_spending',
       'private_health_insurance_spending', 'medicare_spending_by_residence',
       'health_care_spending', 'healthcare_utilization', 'poor_health_status',
       'adults_at_high_risk', 'poverty_rate', 'social_vulnerability_index',
       'Healthcare Access and Quality Index', 'Older_at_high_risk',
       'dem_percent', 'rep_percent'

### Cases

In [113]:
cases = pd.read_pickle(cwd + '/processed_data/confimed_cases_daily_state.pkl')

In [114]:
cases['Week'] = cases['Date'].dt.to_period('W')
cases_week = cases.groupby(['Week', 'state_name', 'fips','Description']).sum().reset_index()
cases_week['Week_start'] = cases_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
cases_week['Week_end'] = cases_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
cases_week = cases_week.rename(columns = {'value' : 'Reported_cases'}).drop(columns = ['Description'])


/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_29365/1334263650.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cases_week = cases.groupby(['Week', 'state_name', 'fips','Description']).sum().reset_index()


In [115]:
for fips in cases_week['fips'].unique():
    index = cases_week[cases_week['fips'] == fips].index
    cases_week.loc[index, 'previous_infection_12w'] = \
    cases_week[cases_week['fips'] == fips]['Reported_cases'].rolling(window = 12, min_periods = 1).sum()

In [116]:
df = hosp_week.merge(cases_week)

In [117]:
df = df.rename(columns = {'Population' : 'population', 'Reported_cases' : 'reported_cases'})
df['reported_cases_per_100k'] = (df['reported_cases']/df['population'])*100000
df['reported_cases_per_100k'] = round(df['reported_cases_per_100k'],1)
df['previous_infection_12w'] = round((df['previous_infection_12w']/df['population'])*100, 1)

### Vaccine 

In [118]:
vacc = pd.read_pickle(cwd + '/processed_data/vaccination_weekly_state.pkl')
vacc['dow'] = vacc['Date'].dt.day_name()

In [119]:
vacc['Week'] = vacc['Date'].dt.to_period('W')
vacc_week = vacc.groupby(['Week', 'state_name', 'fips', 'Variable', 'Description']).mean().reset_index()
vacc_week['Week_start'] = vacc_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
vacc_week['Week_end'] = vacc_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')


/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_29365/314402433.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  vacc_week = vacc.groupby(['Week', 'state_name', 'fips', 'Variable', 'Description']).mean().reset_index()


In [120]:
vacc_week = vacc_week.drop(columns = ['Description'])

In [121]:
df = df.merge(vacc_week[vacc_week['Variable'] == 'Administered_Dose1_Pop_Pct'])
df = df.rename(columns = {'value': 'Dose1_Pop_Pct'}).drop(columns = ['Variable'])
df = df.merge(vacc_week[vacc_week['Variable'] == 'Series_Complete_Pop_Pct'])
df = df.rename(columns = {'value': 'Series_Complete_Pop_Pct'}).drop(columns = ['Variable'])
df = df.merge(vacc_week[vacc_week['Variable'] == 'Additional_Doses_Vax_Pct'])
df = df.rename(columns = {'value': 'Additional_Doses_Vax_Pct'}).drop(columns = ['Variable'])

### Policy

In [122]:
policy = pd.read_pickle(cwd +'/processed_data/selected_state_policy_daily.pkl')

In [123]:
policy['Week'] = policy['Date'].dt.to_period('W')
policy['Week_start'] = policy['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
policy['Week_end'] = policy['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')

In [124]:
policy = policy[pd.to_datetime(policy['Date']) == policy['Week_end']].reset_index().drop(columns = ['index'])

In [125]:
df = df.merge(policy)
df = df.drop(columns = ['population'])
df = df.drop(columns = ['state_abbr', 'Date'])

### Week 3 target

In [128]:
hosp = pd.read_pickle('/Users/hongrudu/Documents/Projects/COVIDLLM/data/processed_data/hospitalization_daily_state.pkl')
hosp['Week'] = hosp['Date'].dt.to_period('W')
hosp_week = hosp.drop(columns = ['Date', 'State', 'Temporal_resolution']).groupby(['state_name', 'Variable', 'fips', 'Week', 'Description']).sum().reset_index()
hosp_week = hosp_week.merge(pop)
hosp_week['hospitalization_per_100k'] = (hosp_week['value']/hosp_week['Population'])*100000
hosp_week = hosp_week.drop(columns = {'Variable', 'Description'})
hosp_week['Week_start'] = hosp_week['Week'].apply(lambda r: r.start_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week['Week_end'] = hosp_week['Week'].apply(lambda r: r.end_time).dt.to_period('D').dt.strftime('%Y-%m-%d')
hosp_week = hosp_week.rename(columns = {'value' : 'hospitalization'})

In [129]:
### Smooth hosp
for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'hospitalization_per_100k_sm'] = \
    hosp_week[hosp_week['fips'] == fips]['hospitalization_per_100k'].rolling(window = 3).mean()

for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'hospitalization_per_100k_sm_lag3'] = \
    hosp_week[hosp_week['fips'] == fips]['hospitalization_per_100k_sm'].shift(3)

hosp_week = hosp_week.dropna().reset_index().drop(columns = ['index'])
hosp_week['Abs_Change_w3'] = hosp_week['hospitalization_per_100k'] - hosp_week['hospitalization_per_100k_sm_lag3']
hosp_week['Abs_Change_w3'] = round(hosp_week['Abs_Change_w3'], 2)


In [130]:
'''
Define threshold
'''
# hosp_week['Change'] = (hosp_week['hospitalization_per_100k'] - hosp_week['hospitalization_per_100k_lag1'])\
# /hosp_week['hospitalization_per_100k_lag1']


index_stable = hosp_week[(hosp_week['Abs_Change_w3'] < 1.5)&(hosp_week['Abs_Change_w3'] >= -1.5)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Stable'
index_stable = hosp_week[(hosp_week['Abs_Change_w3'] >= 1.5)&(hosp_week['Abs_Change_w3'] < 4.5)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Moderate Increasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w3'] >= 4.5)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Substantial Increasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w3'] > -4.5)&(hosp_week['Abs_Change_w3'] <= -1.5)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Moderate Decreasing'
index_stable = hosp_week[(hosp_week['Abs_Change_w3'] <= -4.5)].index
hosp_week.loc[index_stable, 'Abs_Trend'] = 'Substantial Decreasing'


In [131]:
for index, row in hosp_week[hosp_week['Week_end'] >= '2021-01-01'].iterrows():
    prompt = "The most recent hospitalization trends are: " + hosp_week.loc[index-4, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-3, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-2, 'Abs_Trend'].upper() +\
    ', ' + hosp_week.loc[index-1, 'Abs_Trend'].upper() + ', and ' + hosp_week.loc[index, 'Abs_Trend'].upper()
    hosp_week.loc[index, 'Trend_prompt'] = prompt

In [132]:
for fips in hosp_week['fips'].unique():
    index = hosp_week[hosp_week['fips'] == fips].index
    hosp_week.loc[index, 'Abs_Trend_3w'] = hosp_week.loc[index, 'Abs_Trend'].shift(-3)


In [133]:
hosp_week = hosp_week.rename(columns = {'Abs_Trend_3w' : 'Abs_Trend_target_3w'})

In [134]:
df = df.merge(hosp_week[['state_name', 'fips', 'Week', 'Week_start', 'Week_end', 'Abs_Trend_target_3w']])

In [136]:
df.to_pickle(cwd + '/processed_data/all_data.pkl')